## $\textbf{Question 4}$

$\text{Soit le système}$

\begin{align*}
    x_1 + x_2 \leq 1&\\
    x_1 + x_2 \geq 2&\\
    x_1, x_2 \geq 0&\\
\end{align*}

$\text{Utilisons la phase I du simplexe pour montrer que ce système de possède pas de solution}$

$\text{On converti le problème sous forme standard à l'introduction de deux variables artificiels}$ $x_3,$ $x_4,$ $\text{on a;}$

\begin{array}{ccccc}
    \min\ &&&x_3  &+x_4\\
    \text{tel que}&x_1  &+x_2  &+x_3 &&= 1\\
    &x_1  &+x_2  &&+x_4 &= 2\\
    &x_1, &x_2, &x_3, &x_4 &\geq 0\\
\end{array}

$\text{On obtient le problème sous forme de tableau}$
\begin{array}{ccccc}
    &x_1 & x_2 & x_3 &x_4 & b\\
    &1   & 1   & 1   & 0  & 1\\
    &1   & 1   & 0   & 1  & 2\\
    c^T& 0   &0 &1 &1 &0\\
\end{array}
$\text{Pour obtenir une solution de base réalisable, il faut annuler les
coefficients de}$ $x_3$ $\text{et}$ $x_4.$ $\text{Nous obtenons le nouveau tableau}$
\begin{array}{ccccc}
    &x_1 & x_2 & x_3 &x_4 & b\\
    &1   & 1   & 1   & 0  & 1\\
    &1   & 1   & 0   & 1  & 2\\
c^T &-2   &-2    &0    &0   &-3\\
\end{array}

$\text{On peut maintenant continuer la résolution du système en utilisant le simplexe, comme d’ordinaire}$

$\text{premier pivotage}$

\begin{array}{ccccc}
    &x_1 & x_2 & x_3 &x_4 & b\\
    &1   & 1   & 1   & 0  & 1\\
    &0   & 0   & -1   & 1  & 1\\
c^T &0   &0    &2    &0   &-1\\
\end{array}

$\text{les colonnes de}$ $x_1$ $\text{et}$ $x_2$ $\text{étant linéairement dépendantes, on a ainsi un cas de dégénéréscence du problème.}$

$\text{La partie en julia;}$

In [1]:
function pivot!(M::Matrix, i::Int, j::Int)
    m, n = size(M)
    @assert M[i, j] != 0
    M[i, :] = M[i, :]/M[i, j]
    for k in setdiff(1:m, i)
        M[k, :] -= M[k, j] * M[i, :]
    end
    return M
end

function getReducedCosts(M::Matrix)
    m, n = size(M)
    return M[end, 1:n-1]
end
function getxB(M::Matrix)
    m, n = size(M)
    return M[1:m-1, end]
end
function enteringVar(M::Matrix)
    rc = getReducedCosts(M)
    index = argmin(rc)
    return rc[index] >= 0 ? -1 : index
end
function exitingVarIndex(M::Matrix{T}, enteringVar::Int) where T
    col = M[1:end-1, enteringVar]
    xB = getxB(M)
    m, n = size(M)
    index = -1
    val = T(Inf)
    for i in 1:m-1
        if (col[i] > 0) && (xB[i]/col[i] < val)
            val = xB[i]/col[i]
            index = i
        end
    end
    return index
end
function isOneHot(v::Vector)
    n = length(v)
    return (sum(iszero, v) == n-1) && (sum(isone, v) == 1)
end 
function isoptimal(M)
    return enteringVar(M) == -1
end
function findInitialBasis!(M::Matrix)
    m, n = size(M)
    m-=1
    n-=1
    basis = [-1 for _ in 1:m]
    for i in 1:n
        if isOneHot(M[1:end-1, i])
            index = findfirst(isone, M[:, i])
            basis[index] = i
        end
    end
    @assert !any(t-> t == -1, basis) "problem not caconical"
    for i in 1:m
        j = basis[i]
        pivot!(M, i, j)
    end
    return basis
end
function simplexSolver(A::Matrix{T}, b::Vector, c::Vector; verbose::Bool = false) where T
    M = [A b; c' 0]
    basis = findInitialBasis!(M)
    k = 1
    nmax = 1000
    while !isoptimal(M) && k < nmax
        k+=1
        verbose && display(M)
        entering = enteringVar(M)
        entering == -1 && (println("------------"); return T[-1, -1])
        leaving = exitingVarIndex(M, entering)
        leaving == -1 && (println("------------"); return T[-1, -1])
        verbose && @show (entering, leaving)
        basis[leaving] = entering
        pivot!(M, leaving, entering)
    end
    verbose && display(M)
    m, n = size(M)
    xstar = zeros(T, n - 1)
    xstar[basis] = getxB(M)
    return xstar
end

simplexSolver (generic function with 1 method)

In [2]:
function  devoir3_4()
    
    A=[1//1 1 1 0 ; 
          1 1 0 1;
    ]
    
    b=[1, 2]
    
    c = [0, 0, 1, 1]
    simplexSolver(A, b, c, verbose=true)
end

devoir3_4()

3×5 Matrix{Rational{Int64}}:
  1//1   1//1  1//1  0//1   1//1
  1//1   1//1  0//1  1//1   2//1
 -2//1  -2//1  0//1  0//1  -3//1

3×5 Matrix{Rational{Int64}}:
 1//1  1//1   1//1  0//1   1//1
 0//1  0//1  -1//1  1//1   1//1
 0//1  0//1   2//1  0//1  -1//1

(entering, leaving) = (1, 1)


4-element Vector{Rational{Int64}}:
 1//1
 0//1
 0//1
 1//1